<a href="https://colab.research.google.com/github/seon0415/weit/blob/main/%E2%98%85%EC%A7%80%EA%B5%AC%EB%8C%80%EC%88%9C%EC%B0%B0%EC%B0%A8_%EC%88%98%20%26%20%EC%B7%A8%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
plt.rc('font', family='Malgun Gothic')

In [ ]:
car=pd.read_csv("경찰청 서울특별시경찰청_경찰서별 112 순찰차 현황_20211231.csv", encoding='CP949')
police=pd.read_csv("경찰청 서울특별시경찰청_경찰서별 지역경찰 현황_20211231.csv", encoding='CP949')
office=pd.read_csv("시내주요기관(경찰·소방관서)_20230317212437.csv")

## **데이터 EDA**

In [ ]:
car['경찰서']=car['경찰서'].str.replace(" ","")
car['경찰서']=car['경찰서']+'구'
police['경찰서']=police['경찰서']+'구'

In [ ]:
office=pd.read_csv("시내주요기관(경찰·소방관서)_20230317212437.csv")
office.rename(columns={'자치구별(2)':'경찰서'}, inplace=True)
office['2021년_경찰,소방관수']=0
office=office.replace('-',0)
office.drop([0,1], axis=0, inplace=True)

for w in office.columns:
  val="2021" not in w
  val2="경찰서" not in w
  if val2 and val:
    office.drop(columns=w, inplace=True)


for w in office.columns:
    val2="경찰서" not in w
    val3="2021년_경찰,소방관수" not in w
    if val2 and val3:
          office[w]=office[w].astype('float')
          office[w]=office[w].astype('int')
          office['2021년_경찰,소방관수']+=office.loc[:,w]
          office.drop(columns=w, inplace=True)

office.rename(columns={'2021년_경찰,소방관수':'경찰서,소방관 수'}, inplace=True)
office

In [ ]:
df1=pd.merge(car,police,on='경찰서').drop(columns='순번')
df1=pd.merge(df1,office, on='경찰서')
# df['2018년_지역경찰']=df['2018년 지구대']+df['2018년 파출소']+df['2018년 치안센터']
# df['2019년_지역경찰']=df['2019년 지구대']+df['2019년 파출소']+df['2019년 치안센터']
# df['2020년_지역경찰']=df['2020년 지구대']+df['2020년 파출소']+df['2020년 치안센터']
df1['지역경찰 수']=df1['2021년 지구대']+df1['2021년 파출소']+df1['2021년 치안센터']
df1.rename(columns={'대수':'순찰차', '경찰서':'자치구별'}, inplace=True)
df1.drop(columns=df1.iloc[:,2:14], inplace=True)


In [ ]:
#경찰관련데이터 면적당으로 바꾸기
ground=pd.read_csv('행정구역.csv')
ground=ground[['자치구별(2)','2021']].rename(columns={'자치구별(2)':'자치구별','2021':'면적넓이'}).drop(index=[0,1,2],axis=1)
ground['면적넓이']=ground['면적넓이'].astype('float')

df2=pd.merge(df1,ground,on='자치구별')
df2['순찰차']=df2['순찰차']/df2['면적넓이']
df2['경찰서,소방관 수']=df2['경찰서,소방관 수']/df2['면적넓이']
df2['지역경찰 수']=df2['지역경찰 수']/df2['면적넓이']

df2.rename(columns={'순찰차':'면적당 순찰차', '경찰서,소방관 수':'면적당 경찰서,소방관 수', '지역경찰 수':'면적당 지역경찰 수'}, inplace=True)
df2

In [ ]:
df2.to_csv(r'경찰데이터.csv',encoding="utf-8-sig", index=False)

## **지도시각화**

In [ ]:
geo_str=json.load(open('skorea-municipalities-2018-geo.json', encoding='utf-8'))

In [ ]:
#순찰차 수
car_map=folium.Map(
        location=[37.566345, 126.977893], zoom_start=10.5
)

folium.Choropleth(
    geo_data= geo_str,
    data=df,
    columns=['지역','순찰차'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(car_map)

car_map

In [ ]:
#경찰 수 
police_map=folium.Map(
            location=[37.566345, 126.977893], zoom_start=10.5
)

folium.Choropleth(
    geo_data= geo_str,
    data=df,
    columns=['지역','지역경찰 수'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(police_map)

police_map

In [ ]:
#경찰청수
office_map=folium.Map(
        location=[37.566345, 126.977893], zoom_start=10.5
)

folium.Choropleth(
    geo_data= geo_str,
    data=df,
    columns=['지역','경찰서,소방관 수'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(office_map)

office_map

## **히트맵 상관관계 분석**

# **합치기**

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')

In [29]:
arch=pd.read_csv('1인가구.csv')
seoul_map=pd.read_csv('행정구역.csv')
seoul_nmap=pd.read_excel('개발제한구역.xlsx')
cctv=pd.read_csv('cctv.csv',encoding='CP949')
family_num=pd.read_csv('가구원수별가구.csv')
economy=pd.read_csv('economy.csv',encoding='CP949')
apartment=pd.read_csv('apartment.csv',encoding='CP949')
ed=pd.read_csv('교육정도.csv')
df2=pd.read_csv('경찰데이터.csv')

crime=pd.DataFrame({'자치구별':['광진구','동대문구','중랑구','마포구','구로구','금천구','동작구','관악구','강남구'],
                    '면적당 성범죄수':[12.192, 11.884, 16.164,13.459,9.045,10.291,11.009,10.855,14.632,]})

/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [30]:
#형식 통일(컬럼이름, 타입)
arch=arch[['자치구별(2)','2021']].rename(columns={'자치구별(2)':'자치구별'}).drop(index=[0, 1, 2,3],axis=1).rename(columns={'2021':'1인가구'}).set_index('자치구별')
arch['1인가구']=arch['1인가구'].astype('float')

cctv=cctv[['gu','one']].rename(columns={'gu':'자치구별','one':'면적당 cctv'}).set_index('자치구별')


family_num=family_num[['동별(2)','2021.8']].rename(columns={'동별(2)':'자치구별', '2021.8':'평균 가구원수'}).drop(index=[0, 1, 2],axis=1).set_index('자치구별')
family_num['평균 가구원수']=family_num['평균 가구원수'].astype('float')

economy=economy[['gu','one']].rename(columns={'gu':'자치구별','one':'면적당 경제수준'}).set_index('자치구별')

apt=apartment.rename(columns={'gu':'자치구별','lot':'apartment'}).set_index('자치구별')

seoul_map=seoul_map[['자치구별(2)','2021.1']].rename(columns={'자치구별(2)':'자치구별','2021.1':'면적 구성비'}).drop(index=[0,1,2],axis=1).set_index('자치구별')
seoul_map['면적 구성비']=seoul_map['면적 구성비'].astype('float')

In [ ]:
print(arch.dtypes, cctv.dtypes, family_num.dtypes, economy.dtypes, apartment.dtypes)

In [31]:
#합치고 인덱스 설정
df=pd.merge(cctv, family_num,on='자치구별')
df=pd.merge(df,arch,on='자치구별')
df=pd.merge(df,economy,on='자치구별')
df=pd.merge(df,apt,on='자치구별')
df=pd.merge(df,df2,on='자치구별')
df=pd.merge(df,ed,on='자치구별')
df=pd.merge(df,seoul_map,on='자치구별')

df.set_index('자치구별',inplace=True)
df
#df=pd.concat([arch,cctv, family_num, economy, apartment], axis=0) 이렇게 하면 결측치로 뜸

,면적당 cctv,평균 가구원수,1인가구,면적당 경제수준,apartment,면적당 순찰차,"면적당 경찰서,소방관 수",면적당 지역경찰 수,단위별 수강률,면적 구성비
자치구별,,,,,,,,,,
종로구,0.075778,2.1,27308.0,1172.498118,999.00,0.794647,1.338352,0.585529,10.971134,3.95
용산구,0.115750,2.1,39270.0,471.435013,1531.37,1.143118,0.868770,0.548697,2.394018,3.61
성동구,0.204105,2.2,44946.0,622.993357,1421.60,1.304864,1.245552,0.889680,3.740473,2.79
광진구,0.193647,2.1,66140.0,349.915368,1369.57,1.641266,0.996483,0.703400,4.852549,2.82
동대문구,0.180373,2.1,65290.0,505.106015,1028.23,2.180028,1.898734,1.406470,13.850285,2.35
중랑구,0.207894,2.2,60487.0,216.985888,796.92,1.297297,1.243243,0.918919,3.436626,3.06
성북구,0.147949,2.3,64985.0,212.288859,1009.58,0.691619,1.261188,0.569569,1.810606,4.06
강북구,0.134915,2.2,48428.0,119.921610,829.92,0.974576,0.847458,0.593220,5.339812,3.90
도봉구,0.088761,2.4,37853.0,136.402644,808.01,0.871671,0.726392,0.435835,8.617375,3.41


In [ ]:
df.to_csv(r'취합.csv',encoding="utf-8-sig")